In [1]:
import numpy as np
import matplotlib.pyplot as plt
import re
import os
from tqdm import tqdm
import sys

sys.path.append(os.path.abspath(".."))

from ae_state_evolution_new import hat_vars_MCMC as hat_vars_MCMC_v1
from ae_state_evolution_correct import hat_vars_MCMC as hat_vars_MCMC_v2


In [3]:

def run_statistical_comparison(alpha, beta, q_vars, gamma, delta, samples=500000, trials=5):
    results_v1 = {"m_hat": [], "q_hat": [], "V_hat": []}
    results_v2 = {"m_hat": [], "q_hat": [], "V_hat": []}

    print(f"Running {trials} trials with {samples} samples each...\n")

    for i in range(trials):
        # We don't reset the seed, allowing natural variation
        
        # Version 1
        m1, q1, V1 = hat_vars_MCMC_v1(alpha, beta, q_vars, samples, gamma, delta)
        results_v1["m_hat"].append(m1)
        results_v1["q_hat"].append(q1)
        results_v1["V_hat"].append(V1)

        # Version 2
        m2, q2, V2 = hat_vars_MCMC_v2(alpha, beta, q_vars, samples, gamma, delta)
        results_v2["m_hat"].append(m2)
        results_v2["q_hat"].append(q2)
        results_v2["V_hat"].append(V2)
        print(f"Trial {i+1}/{trials} complete.")

    print("\n" + "="*80)
    print(f"{'Variable':<10} | {'V1 Mean (StdErr)':<25} | {'V2 Mean (StdErr)':<25} | {'Diff'}")
    print("-" * 80)

    for key in ["m_hat", "q_hat", "V_hat"]:
        # Convert lists to arrays for stats
        v1_arr = np.array(results_v1[key])
        v2_arr = np.array(results_v2[key])
        
        v1_mean, v1_err = np.mean(v1_arr, axis=0), np.std(v1_arr, axis=0) / np.sqrt(trials)
        v2_mean, v2_err = np.mean(v2_arr, axis=0), np.std(v2_arr, axis=0) / np.sqrt(trials)
        
        diff = np.linalg.norm(v1_mean - v2_mean)
        
        # Formatting for vectors (m_hat) vs scalars
        v1_str = f"{np.round(v1_mean, 5)} ({np.round(v1_err, 5)})"
        v2_str = f"{np.round(v2_mean, 5)} ({np.round(v2_err, 5)})"
        
        print(f"{key:<10} | {v1_str:<25} | {v2_str:<25} | {diff:.2e}")

# --- Execution ---
if __name__ == "__main__":
    # Ensure these variables match your physical setup
    alpha_test = 20.0
    beta_test = (1.0, 2.0)
    m_test = np.array([[0.6, 0.1]])
    q_test = 1.0
    V_test = 0.3
    q_vars_test = (m_test, q_test, V_test)
    
    run_statistical_comparison(
        alpha=alpha_test, 
        beta=beta_test, 
        q_vars=q_vars_test, 
        gamma=1.0, 
        delta=0.5, 
        samples=500000, 
        trials=5
    )

Running 5 trials with 500000 samples each...

Trial 1/5 complete.
Trial 2/5 complete.
Trial 3/5 complete.
Trial 4/5 complete.
Trial 5/5 complete.

Variable   | V1 Mean (StdErr)          | V2 Mean (StdErr)          | Diff
--------------------------------------------------------------------------------
m_hat      | [[9.46874 5.56976]] ([[0.02281 0.00943]]) | [[9.50817 5.58905]] ([[0.03068 0.01374]]) | 4.39e-02
q_hat      | 29.24409 (0.07319)        | 29.33934 (0.04211)        | 9.53e-02
V_hat      | 15.01145 (0.03657)        | 17.61209 (0.02307)        | 2.60e+00
